# USE OF OPTIMIZATION TECHNIQUES

## PREPARING THE DEVELOPMENT ENVIRONMENT

Including path to previous directory in built-in variable `sys.path`, allowing access to the `sudoku` package:

In [1]:
import sys

sys.path.append('../')

Importing the relevant libraries:

In [2]:
import numpy as np
from itertools import product
from mip import Model, MINIMIZE, CBC, BINARY, xsum
from sudoku.utils import N, STEP, process_grid, objective_grid

pygame 2.5.2 (SDL 2.28.3, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


## MODELING SUDOKU AS AN INTEGER PROGRAMMING PROBLEM

Documentation explaining the model below can be found in the [project repository](https://github.com/filipemedeiross/solving_sudoku_by_search_for_csp):

In [3]:
def create_model(grid):
    model = Model(sense=MINIMIZE, solver_name=CBC)

    V = set(range(N))
    S = set(range(0, N, STEP))

    x = [[[model.add_var(var_type=BINARY, name=f'x_{i}{j}{k}')
           for k in range(1, N+1)]
           for j in range(N)]
           for i in range(N)]

    # Game clues must be strictly followed
    for i, j, k in grid:
        if k:
            model += x[i][j][k-1] == 1, f'CLUE_x_{i}{j}{k}'

    # All cells must be filled in necessarily once
    for i, j in product(V, V):
        model += xsum(x[i][j][k] for k in range(N)) == 1, f'FILL_x_{i}{j}'

    # AllDiff constraints on rows
    for i, k in product(V, V):
        model += xsum(x[i][j][k] for j in range(N)) == 1, f'ALLDIFF_row_{i}{k}'

    # AllDiff constraints on cols
    for j, k in product(V, V):
        model += xsum(x[i][j][k] for i in range(N)) == 1, f'ALLDIFF_col_{j}{k}'

    # AllDiff constraints on squares
    for m, n, k in product(S, S, V):
        model += xsum(x[i][j][k]
                      for i in range(m, m+STEP)
                      for j in range(n, n+STEP)) == 1, f'ALLDIFF_square_{m}{n}{k}'

    return model, x, V

In [4]:
def solver_ip(grid):
    grid  = grid.copy()
    entry = process_grid(grid)

    model, var, V = create_model(entry)
    model.verbose = 0
    model.optimize()

    for i, j, k in product(V, V, V):
        if var[i][j][k].x and not grid[i, j]:
            grid[i, j] = k + 1

    return grid

## TESTING THE SOLVER

Checking the efficiency of the model:

In [5]:
%%timeit

grid = np.array([[0, 0, 5, 3, 0, 0, 0, 0, 0],
                 [8, 0, 0, 0, 0, 0, 0, 2, 0],
                 [0, 7, 0, 0, 1, 0, 5, 0, 0],
                 [4, 0, 0, 0, 0, 5, 3, 0, 0],
                 [0, 1, 0, 0, 7, 0, 0, 0, 6],
                 [0, 0, 3, 2, 0, 0, 0, 8, 0],
                 [0, 6, 0, 5, 0, 0, 0, 0, 9],
                 [0, 0, 4, 0, 0, 0, 0, 3, 0],
                 [0, 0, 0, 0, 0, 9, 7, 0, 0]], dtype='int8')

solver_ip(grid)

1.16 s ± 639 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Checking if the objective grid was reached:

In [6]:
grid = np.array([[0, 0, 5, 3, 0, 0, 0, 0, 0],
                 [8, 0, 0, 0, 0, 0, 0, 2, 0],
                 [0, 7, 0, 0, 1, 0, 5, 0, 0],
                 [4, 0, 0, 0, 0, 5, 3, 0, 0],
                 [0, 1, 0, 0, 7, 0, 0, 0, 6],
                 [0, 0, 3, 2, 0, 0, 0, 8, 0],
                 [0, 6, 0, 5, 0, 0, 0, 0, 9],
                 [0, 0, 4, 0, 0, 0, 0, 3, 0],
                 [0, 0, 0, 0, 0, 9, 7, 0, 0]], dtype='int8')

goal_grid = solver_ip(grid)

print(f'IP was sufficient to resolve the instance of the problem = {objective_grid(goal_grid)}')

print()

print('Grid = ')
print(goal_grid)

IP was sufficient to resolve the instance of the problem = True

Grid = 
[[1 4 5 3 2 7 6 9 8]
 [8 3 9 6 5 4 1 2 7]
 [6 7 2 9 1 8 5 4 3]
 [4 9 6 1 8 5 3 7 2]
 [2 1 8 4 7 3 9 5 6]
 [7 5 3 2 9 6 4 8 1]
 [3 6 7 5 4 2 8 1 9]
 [9 8 4 7 6 1 2 3 5]
 [5 2 1 8 3 9 7 6 4]]
